# Lab | Comparing regression models

For this lab, we will be using the same dataset we used in the previous labs. We recommend using the same notebook since you will be reusing the same variables you previous created and used in labs.

## Instructions
1. In this final lab, we will model our data. 
Import sklearn train_test_split and separate the data.

2. We will start with removing outliers, if you have not already done so. We have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.

3. Create a copy of the dataframe for the data wrangling.

4. Normalize the continuous variables. You can use any one method you want.

5. Encode the categorical variables (See the hint below for encoding categorical data!!!)

6. The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.

7. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

## Hint for Categorical Variables
You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):

Encoder Type	Column
One hot	state
Ordinal	coverage
Ordinal	employmentstatus
Ordinal	location code
One hot	marital status
One hot	policy type
One hot	policy
One hot	renew offercustomer_df
One hot	sales channel
One hot	vehicle class
Ordinal	vehicle size

#### Dummy code
data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})

given that column "coverage" in the dataframe "data" has three categories:

"basic", "extended", and "premium" and values are to be represented in the same order.

8. Try a simple linear regression with all the data to see whether we are getting good results.

9. Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

10.Use the function to check LinearRegressor and KNeighborsRegressor.

11. You can check also the MLPRegressor for this task!

12. Check and discuss the results.

# Lab | Comparing regression models

For this lab, we will be using the same dataset we used in the previous labs. We recommend using the same notebook since you will be reusing the same variables you previous created and used in labs.

## Instructions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score

In [2]:
customer = pd.read_csv("we_fn_use_c_marketing_customer_value_analysis.csv")
display(customer.head())
customer.info()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer                       9134 non-null   object 
 1   State                          9134 non-null   object 
 2   Customer Lifetime Value        9134 non-null   float64
 3   Response                       9134 non-null   object 
 4   Coverage                       9134 non-null   object 
 5   Education                      9134 non-null   object 
 6   Effective To Date              9134 non-null   object 
 7   EmploymentStatus               9134 non-null   object 
 8   Gender                         9134 non-null   object 
 9   Income                         9134 non-null   int64  
 10  Location Code                  9134 non-null   object 
 11  Marital Status                 9134 non-null   object 
 12  Monthly Premium Auto           9134 non-null   i

I have taken the liberty of reversing the order of the first two questions.

We are now going to make some changes already applied in the 2 previous labs.

## 0. Cleaning the data

In [3]:
#lowercase and removing spaces
columns = []
for col in customer.columns:
    col = col.lower().replace(" ", "_")
    columns.append(col)
    
customer.columns = columns
customer

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA72316,California,23405.987980,No,Basic,Bachelor,2/10/11,Employed,M,71941,...,89,0,2,Personal Auto,Personal L1,Offer2,Web,198.234764,Four-Door Car,Medsize
9130,PK87824,California,3096.511217,Yes,Extended,College,2/12/11,Employed,F,21604,...,28,0,1,Corporate Auto,Corporate L3,Offer1,Branch,379.200000,Four-Door Car,Medsize
9131,TD14365,California,8163.890428,No,Extended,Bachelor,2/6/11,Unemployed,M,0,...,37,3,2,Corporate Auto,Corporate L2,Offer1,Branch,790.784983,Four-Door Car,Medsize
9132,UP19263,California,7524.442436,No,Extended,College,2/3/11,Employed,M,21941,...,3,0,3,Personal Auto,Personal L2,Offer3,Branch,691.200000,Four-Door Car,Large


In [4]:
#changing the tipe of the column 'effective_to_date' to datetime
customer['effective_to_date']=pd.to_datetime(customer['effective_to_date'])
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   customer                       9134 non-null   object        
 1   state                          9134 non-null   object        
 2   customer_lifetime_value        9134 non-null   float64       
 3   response                       9134 non-null   object        
 4   coverage                       9134 non-null   object        
 5   education                      9134 non-null   object        
 6   effective_to_date              9134 non-null   datetime64[ns]
 7   employmentstatus               9134 non-null   object        
 8   gender                         9134 non-null   object        
 9   income                         9134 non-null   int64         
 10  location_code                  9134 non-null   object        
 11  marital_status   

In [5]:
#Checking for Nans
customer.isna().sum()

customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
dtype: int64

In [6]:
#Change name of column "employment status"
customer = customer.rename(columns={"employmentstatus":"employment_status"})
customer

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2011-02-24,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,2011-01-31,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2011-02-19,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,2011-01-20,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2011-02-03,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA72316,California,23405.987980,No,Basic,Bachelor,2011-02-10,Employed,M,71941,...,89,0,2,Personal Auto,Personal L1,Offer2,Web,198.234764,Four-Door Car,Medsize
9130,PK87824,California,3096.511217,Yes,Extended,College,2011-02-12,Employed,F,21604,...,28,0,1,Corporate Auto,Corporate L3,Offer1,Branch,379.200000,Four-Door Car,Medsize
9131,TD14365,California,8163.890428,No,Extended,Bachelor,2011-02-06,Unemployed,M,0,...,37,3,2,Corporate Auto,Corporate L2,Offer1,Branch,790.784983,Four-Door Car,Medsize
9132,UP19263,California,7524.442436,No,Extended,College,2011-02-03,Employed,M,21941,...,3,0,3,Personal Auto,Personal L2,Offer3,Branch,691.200000,Four-Door Car,Large


## 2. We will start with removing outliers, if you have not already done so.
We have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.

In [7]:
iqr = np.nanpercentile(customer['customer_lifetime_value'],75) - np.nanpercentile(customer['customer_lifetime_value'],25)
upper_limit = np.nanpercentile(customer['customer_lifetime_value'],75) + 1.5*iqr
customer_df = customer[customer['customer_lifetime_value'] < upper_limit]



In [8]:
iqr = np.nanpercentile(customer['total_claim_amount'],75) - np.nanpercentile(customer['total_claim_amount'],25)
upper_limit = np.nanpercentile(customer['total_claim_amount'],75) + 1.5*iqr 
customer = customer[customer['total_claim_amount'] < upper_limit]

In [9]:
customer_df.shape

(8317, 24)

## 1. In this final lab, we will model our data. 
Import sklearn train_test_split and separate the data.

In [10]:
X = customer.drop(["customer", "total_claim_amount"], axis = 1)
y = customer["total_claim_amount"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 25)

print(X_train.shape)
print(X_test.shape)

(6510, 22)
(2171, 22)


## 3. Create a copy of the dataframe for the data wrangling.

In [12]:
X_train_num = X_train.select_dtypes(np.number)
X_train_cat = X_train.select_dtypes("object")

X_test_num = X_test.select_dtypes(np.number)
X_test_cat = X_test.select_dtypes("object")

## 4. Normalize the continuous variables. 
You can use any one method you want.

In [13]:
#We are going to use the logaritmic transformation. 
#To do so, we need the number to be both finite and diferent to 0.
#We are going to apply it to the 'customer_lifetime_value' and 'income' columns.

def log_transform(x):
    if (np.isfinite(x)) and (x != 0):
        return np.log(x)
    else:
        return x

#Then we apply it to both train and test

X_train_num["customer_lifetime_value"] = X_train_num["customer_lifetime_value"].apply(log_transform)
X_train_num["income"] = X_train_num["income"].apply(log_transform)

X_test_num["customer_lifetime_value"] = X_test_num["customer_lifetime_value"].apply(log_transform)
X_test_num["income"] = X_test_num["income"].apply(log_transform)

In [14]:
mm_scaler = MinMaxScaler().fit(X_train_num)
X_train_num_mm = mm_scaler.transform(X_train_num)
X_train_num_mm = pd.DataFrame(X_train_num_mm, columns = X_train_num.columns)

X_test_num_mm = mm_scaler.transform(X_test_num)
X_test_num_mm = pd.DataFrame(X_test_num_mm, columns = X_test_num.columns)

In [15]:
X_train_num_mm

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0.211746,0.959713,0.182979,0.571429,0.141414,0.0,0.000
1,0.271570,0.973655,0.272340,0.142857,0.575758,0.0,0.000
2,0.099481,0.000000,0.051064,0.400000,0.666667,0.0,0.000
3,0.368498,0.872496,0.119149,0.857143,0.222222,0.0,0.750
4,0.248813,0.000000,0.306383,0.514286,0.020202,0.0,0.000
...,...,...,...,...,...,...,...
6505,0.635759,0.971758,0.021277,0.628571,0.232323,0.0,0.125
6506,0.663392,0.000000,0.051064,0.000000,0.212121,0.2,0.125
6507,0.580050,0.935781,0.008511,0.285714,0.666667,0.0,0.125
6508,0.464868,0.974451,0.268085,0.314286,0.363636,0.0,0.250


## 5. Encode the categorical variables 
(See the hint below for encoding categorical data!!!)

In [16]:
X_train_cat

,state,response,coverage,education,employment_status,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
5664,Washington,No,Premium,Bachelor,Employed,F,Rural,Single,Personal Auto,Personal L2,Offer3,Agent,Two-Door Car,Small
5700,California,No,Extended,College,Employed,F,Urban,Married,Special Auto,Special L1,Offer1,Branch,SUV,Medsize
7712,Nevada,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Personal Auto,Personal L1,Offer2,Agent,Four-Door Car,Medsize
1739,Arizona,No,Extended,Bachelor,Employed,F,Suburban,Married,Corporate Auto,Corporate L1,Offer1,Branch,Four-Door Car,Medsize
4141,Oregon,No,Extended,High School or Below,Unemployed,M,Suburban,Single,Personal Auto,Personal L3,Offer1,Web,SUV,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Arizona,No,Basic,College,Employed,M,Urban,Married,Personal Auto,Personal L3,Offer2,Web,Four-Door Car,Medsize
8886,California,No,Basic,Bachelor,Unemployed,F,Suburban,Single,Corporate Auto,Corporate L2,Offer1,Branch,Four-Door Car,Medsize
3073,Nevada,No,Basic,Doctor,Employed,M,Rural,Married,Personal Auto,Personal L1,Offer2,Call Center,Two-Door Car,Medsize
6969,California,No,Extended,Bachelor,Employed,F,Rural,Married,Special Auto,Special L2,Offer3,Branch,SUV,Medsize


In [17]:
X_train_cat_oh = X_train_cat[["state", "response", "gender", "marital_status", "policy_type", "sales_channel", "vehicle_class"]]
X_train_cat_ord = X_train_cat.drop(columns = ["state", "response", "gender", "marital_status", "policy_type", "sales_channel", "vehicle_class"], axis = 1)

X_test_cat_oh = X_test_cat[["state", "response", "gender", "marital_status", "policy_type", "sales_channel", "vehicle_class"]]
X_test_cat_ord = X_test_cat.drop(columns = ["state", "response", "gender", "marital_status", "policy_type", "sales_channel", "vehicle_class"], axis = 1)

In [18]:
X_train_cat_oh = pd.get_dummies(X_train_cat_oh, drop_first = True)
X_test_cat_oh = pd.get_dummies(X_test_cat_oh, drop_first = True)
X_train_cat_oh

,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
5664,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1
5700,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0
7712,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0
1739,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4141,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0
8886,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
3073,0,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1
6969,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0


In [19]:
X_train_cat_ord

,coverage,education,employment_status,location_code,policy,renew_offer_type,vehicle_size
5664,Premium,Bachelor,Employed,Rural,Personal L2,Offer3,Small
5700,Extended,College,Employed,Urban,Special L1,Offer1,Medsize
7712,Basic,Bachelor,Unemployed,Suburban,Personal L1,Offer2,Medsize
1739,Extended,Bachelor,Employed,Suburban,Corporate L1,Offer1,Medsize
4141,Extended,High School or Below,Unemployed,Suburban,Personal L3,Offer1,Medsize
...,...,...,...,...,...,...,...
1227,Basic,College,Employed,Urban,Personal L3,Offer2,Medsize
8886,Basic,Bachelor,Unemployed,Suburban,Corporate L2,Offer1,Medsize
3073,Basic,Doctor,Employed,Rural,Personal L1,Offer2,Medsize
6969,Extended,Bachelor,Employed,Rural,Special L2,Offer3,Medsize


In [20]:
X_train_cat_ord["coverage"].unique()

array(['Premium', 'Extended', 'Basic'], dtype=object)

In [21]:
X_train_cat_ord["coverage"] = X_train_cat_ord["coverage"].map({"Basic":0, "Extended":0.5, "Premium":1})
X_test_cat_ord["coverage"] = X_test_cat_ord["coverage"].map({"Basic":0, "Extended":0.5, "Premium":1})

In [22]:
X_train_cat_ord["education"].unique()

array(['Bachelor', 'College', 'High School or Below', 'Doctor', 'Master'],
      dtype=object)

In [23]:
X_train_cat_ord["education"] = X_train_cat_ord["education"].map({"High School or Below":0, "University":0.5, "Postgraduate":1})
X_test_cat_ord["education"] = X_test_cat_ord["education"].map({"High School or Below":0, "University":0.5, "Postgraduate":1})

In [24]:
X_train_cat_ord["employment_status"].unique()

array(['Employed', 'Unemployed', 'Disabled', 'Medical Leave', 'Retired'],
      dtype=object)

In [25]:
X_train_cat_ord["employment_status"] = X_train_cat_ord["employment_status"].map({"Excempt":0, "Unemployed":0.5, "Employed":1})
X_test_cat_ord["employment_status"] = X_test_cat_ord["employment_status"].map({"Excempt":0, "Unemployed":0.5, "Employed":1})

In [26]:
X_train_cat_ord["location_code"].unique()

array(['Rural', 'Urban', 'Suburban'], dtype=object)

In [27]:
X_train_cat_ord["location_code"] = X_train_cat_ord["location_code"].map({"Rural":0, "Suburban":0.5, "Urban":1})
X_test_cat_ord["location_code"] = X_test_cat_ord["location_code"].map({"Rural":0, "Suburban":0.5, "Urban":1})

In [28]:
X_train_cat_ord["policy"].unique()

array(['Personal L2', 'Special L1', 'Personal L1', 'Corporate L1',
       'Personal L3', 'Corporate L2', 'Corporate L3', 'Special L3',
       'Special L2'], dtype=object)

In [29]:
X_train_cat_ord["policy"] = X_train_cat_ord["policy"].map({"L1":0, "L2":0.5, "L3":1})
X_test_cat_ord["policy"] = X_test_cat_ord["policy"].map({"L1":0, "L2":0.5, "L3":1})

In [30]:
X_train_cat_ord["vehicle_size"].unique()

array(['Small', 'Medsize', 'Large'], dtype=object)

In [31]:
X_train_cat_ord["vehicle_size"] = X_train_cat_ord["vehicle_size"].map({"Small":0, "Medsize":0.5, "Large":1})
X_test_cat_ord["vehicle_size"] = X_test_cat_ord["vehicle_size"].map({"Small":0, "Medsize":0.5, "Large":1})

In [32]:
X_train_cat_oh


,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
5664,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1
5700,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0
7712,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0
1739,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4141,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0
8886,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
3073,0,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1
6969,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0


In [33]:
X_train_cat_ord

,coverage,education,employment_status,location_code,policy,renew_offer_type,vehicle_size
5664,1.0,NaN,1.0,0.0,NaN,Offer3,0.0
5700,0.5,NaN,1.0,1.0,NaN,Offer1,0.5
7712,0.0,NaN,0.5,0.5,NaN,Offer2,0.5
1739,0.5,NaN,1.0,0.5,NaN,Offer1,0.5
4141,0.5,0.0,0.5,0.5,NaN,Offer1,0.5
...,...,...,...,...,...,...,...
1227,0.0,NaN,1.0,1.0,NaN,Offer2,0.5
8886,0.0,NaN,0.5,0.5,NaN,Offer1,0.5
3073,0.0,NaN,1.0,0.0,NaN,Offer2,0.5
6969,0.5,NaN,1.0,0.0,NaN,Offer3,0.5


In [34]:
X_train_cat_encoded = pd.concat([X_train_cat_oh, X_train_cat_ord], axis = 1).reset_index()
X_test_cat_encoded = pd.concat([X_test_cat_oh, X_test_cat_ord], axis = 1).reset_index()

X_train_cat_encoded = X_train_cat_encoded.drop(columns = X_train_cat_encoded.columns[0], axis = 1)
X_test_cat_encoded = X_test_cat_encoded.drop(columns = X_test_cat_encoded.columns[0], axis = 1)
X_train_cat_encoded

,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,...,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,coverage,education,employment_status,location_code,policy,renew_offer_type,vehicle_size
0,0,0,0,1,0,0,0,1,1,0,...,0,0,1,1.0,NaN,1.0,0.0,NaN,Offer3,0.0
1,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0.5,NaN,1.0,1.0,NaN,Offer1,0.5
2,0,1,0,0,0,1,1,0,1,0,...,0,0,0,0.0,NaN,0.5,0.5,NaN,Offer2,0.5
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0.5,NaN,1.0,0.5,NaN,Offer1,0.5
4,0,0,1,0,0,1,0,1,1,0,...,1,0,0,0.5,0.0,0.5,0.5,NaN,Offer1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6505,0,0,0,0,0,1,1,0,1,0,...,0,0,0,0.0,NaN,1.0,1.0,NaN,Offer2,0.5
6506,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0.0,NaN,0.5,0.5,NaN,Offer1,0.5
6507,0,1,0,0,0,1,1,0,1,0,...,0,0,1,0.0,NaN,1.0,0.0,NaN,Offer2,0.5
6508,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0.5,NaN,1.0,0.0,NaN,Offer3,0.5


## 6. The time variable can be useful.


Try to transform its data into a useful one.

Hint: Day week and month as integers might be useful.

Moved to the beginning to use the MinMaxScaler.


## 7. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

Hint for Categorical Variables
You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):

Dummy code
data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
given that column "coverage" in the dataframe "data" has three categories:

"basic", "extended", and "premium" and values are to be represented in the same order.

In [35]:
X_train_num_mm

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0.211746,0.959713,0.182979,0.571429,0.141414,0.0,0.000
1,0.271570,0.973655,0.272340,0.142857,0.575758,0.0,0.000
2,0.099481,0.000000,0.051064,0.400000,0.666667,0.0,0.000
3,0.368498,0.872496,0.119149,0.857143,0.222222,0.0,0.750
4,0.248813,0.000000,0.306383,0.514286,0.020202,0.0,0.000
...,...,...,...,...,...,...,...
6505,0.635759,0.971758,0.021277,0.628571,0.232323,0.0,0.125
6506,0.663392,0.000000,0.051064,0.000000,0.212121,0.2,0.125
6507,0.580050,0.935781,0.008511,0.285714,0.666667,0.0,0.125
6508,0.464868,0.974451,0.268085,0.314286,0.363636,0.0,0.250


In [36]:
X_train_cat_encoded


,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,...,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,coverage,education,employment_status,location_code,policy,renew_offer_type,vehicle_size
0,0,0,0,1,0,0,0,1,1,0,...,0,0,1,1.0,NaN,1.0,0.0,NaN,Offer3,0.0
1,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0.5,NaN,1.0,1.0,NaN,Offer1,0.5
2,0,1,0,0,0,1,1,0,1,0,...,0,0,0,0.0,NaN,0.5,0.5,NaN,Offer2,0.5
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0.5,NaN,1.0,0.5,NaN,Offer1,0.5
4,0,0,1,0,0,1,0,1,1,0,...,1,0,0,0.5,0.0,0.5,0.5,NaN,Offer1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6505,0,0,0,0,0,1,1,0,1,0,...,0,0,0,0.0,NaN,1.0,1.0,NaN,Offer2,0.5
6506,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0.0,NaN,0.5,0.5,NaN,Offer1,0.5
6507,0,1,0,0,0,1,1,0,1,0,...,0,0,1,0.0,NaN,1.0,0.0,NaN,Offer2,0.5
6508,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0.5,NaN,1.0,0.0,NaN,Offer3,0.5


In [37]:
X_train_treated = pd.concat([X_train_num_mm, X_train_cat_encoded], axis = 1)
X_test_treated = pd.concat([X_test_num_mm, X_test_cat_encoded], axis = 1)

In [38]:
X_train_treated

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_California,state_Nevada,state_Oregon,...,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,coverage,education,employment_status,location_code,policy,renew_offer_type,vehicle_size
0,0.211746,0.959713,0.182979,0.571429,0.141414,0.0,0.000,0,0,0,...,0,0,1,1.0,NaN,1.0,0.0,NaN,Offer3,0.0
1,0.271570,0.973655,0.272340,0.142857,0.575758,0.0,0.000,1,0,0,...,1,0,0,0.5,NaN,1.0,1.0,NaN,Offer1,0.5
2,0.099481,0.000000,0.051064,0.400000,0.666667,0.0,0.000,0,1,0,...,0,0,0,0.0,NaN,0.5,0.5,NaN,Offer2,0.5
3,0.368498,0.872496,0.119149,0.857143,0.222222,0.0,0.750,0,0,0,...,0,0,0,0.5,NaN,1.0,0.5,NaN,Offer1,0.5
4,0.248813,0.000000,0.306383,0.514286,0.020202,0.0,0.000,0,0,1,...,1,0,0,0.5,0.0,0.5,0.5,NaN,Offer1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6505,0.635759,0.971758,0.021277,0.628571,0.232323,0.0,0.125,0,0,0,...,0,0,0,0.0,NaN,1.0,1.0,NaN,Offer2,0.5
6506,0.663392,0.000000,0.051064,0.000000,0.212121,0.2,0.125,1,0,0,...,0,0,0,0.0,NaN,0.5,0.5,NaN,Offer1,0.5
6507,0.580050,0.935781,0.008511,0.285714,0.666667,0.0,0.125,0,1,0,...,0,0,1,0.0,NaN,1.0,0.0,NaN,Offer2,0.5
6508,0.464868,0.974451,0.268085,0.314286,0.363636,0.0,0.250,1,0,0,...,1,0,0,0.5,NaN,1.0,0.0,NaN,Offer3,0.5


In [39]:
X_test_treated

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_California,state_Nevada,state_Oregon,...,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,coverage,education,employment_status,location_code,policy,renew_offer_type,vehicle_size
0,0.364643,0.949473,0.114894,0.857143,0.191919,0.0,0.625,0,1,0,...,0,0,0,0.5,NaN,1.0,1.0,NaN,Offer2,0.0
1,0.490411,0.876663,0.323404,0.057143,0.404040,0.0,0.500,1,0,0,...,0,1,0,0.5,NaN,1.0,0.5,NaN,Offer2,0.5
2,0.515702,0.000000,0.246809,0.085714,0.242424,0.0,0.125,0,0,0,...,0,0,0,1.0,0.0,0.5,0.5,NaN,Offer2,0.5
3,0.089434,0.962863,0.017021,0.828571,0.272727,0.0,0.000,0,0,1,...,0,0,1,0.0,NaN,1.0,0.0,NaN,Offer2,1.0
4,0.416867,0.888507,0.200000,0.085714,0.030303,0.0,0.750,0,0,1,...,1,0,0,0.0,NaN,1.0,0.5,NaN,Offer3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166,0.242183,0.988452,0.217021,0.028571,0.282828,0.2,0.000,0,0,1,...,0,0,0,1.0,NaN,1.0,0.5,NaN,Offer2,0.0
2167,0.395892,0.962002,0.017021,0.028571,0.848485,0.0,0.125,0,0,0,...,0,0,0,0.0,NaN,1.0,1.0,NaN,Offer2,0.5
2168,0.305092,0.000000,0.093617,0.000000,0.121212,0.0,0.750,1,0,0,...,0,0,0,0.5,NaN,0.5,0.5,NaN,Offer3,0.5
2169,0.114481,1.000017,0.042553,0.057143,0.222222,0.0,0.000,1,0,0,...,0,0,0,0.0,NaN,1.0,1.0,NaN,Offer2,0.0


## 8. Try a simple linear regression with all the data to see whether we are getting good results.


In [40]:
lr = LinearRegression()
lr.fit(X_train_treated, y_train)

y_pred_train = lr.predict(X_train_treated)
y_pred_test = lr.predict(X_test_treated)

print(r2_score(y_train, y_pred_train))
print(r2_score(y_test, y_pred_test))

ValueError: could not convert string to float: 'Offer3'

There's a problem that I cannot identify how to solve.

## 9. Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

In [41]:
def model_test(models, X_train, y_train, X_test, y_test):
    for model in models:
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        print("Train score of model {} is {:.2f}".format(model, r2_score(y_train, y_pred_train)))
        print("Test score of model {} is {:.2f}".format(model, r2_score(y_test, y_pred_test)))

## 10. Use the function to check LinearRegressor and KNeighborsRegressor.

In [42]:
models = []

for x in range(2,10):
    knn_model = KNeighborsRegressor(n_neighbors = x)
    models.append(knn_model)

## 11. You can check also the MLPRegressor for this task!

In [43]:
mlp1 = MLPRegressor(random_state = 22, solver = "lbfgs")
mlp2 = MLPRegressor(random_state = 22, solver = "adam")

In [44]:
models.append(mlp1)
models.append(mlp2)
models.append(lr)

In [45]:
model_test(models, X_train_treated, y_train, X_test_treated, y_test)


ValueError: could not convert string to float: 'Offer3'

## 12. Check and discuss the results.